In [16]:
%reset -fs

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

import re



In [17]:
df = pd.read_csv('eu_tweet_df_eda1.csv')

/var/folders/gc/kgfpjt4x1d5c1thdk2sl_2hr0000gn/T/ipykernel_19458/3349357611.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('eu_tweet_df_eda1.csv')


In [18]:
def strip_punc_lower_http(txt):
    txt = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", txt)

    txt = re.sub(pattern= "[^\w\s]",
            repl="",
            string=txt).lower()
    
    txt = re.sub(pattern= "[0-9]",
            repl="",
            string=txt)
    
    return txt
strip_punc_lower_http('I am, testing #ALEX > than X 10. https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html')

'i am testing alex  than x   '

In [19]:
df['text'] = df['text'].apply(lambda x: strip_punc_lower_http(x))

In [20]:
df.head()

,id,created_at,text,user_name,user_screen_name,user_id,user_description,user_location,user_verified,lang,...,in_reply_to_user_id,retweet_count,retweet_id,retweet_screen_name,user_created_at,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
0,866239077045207040,Sun May 21 10:27:44 +0000 2017,pounding the streets of redland this morning w...,Green💚🌻Molly,GreenPartyMolly,726372601,Molly Scott Cato 👋 Green Party External Comms ...,South West,False,en,...,NaN,23.0,8.662375e+17,bristol_yg,Mon Jul 30 16:21:34 +0000 2012,27154.0,59652.0,1965.0,854.0,69671.0
1,866232717196840960,Sun May 21 10:02:28 +0000 2017,the ukip battle bus is coming please support m...,Dr Julia Reid PhD,julia_reid,43554444,Former Brexit Party MEP for the SW (2014-19) -...,Calne,True,en,...,NaN,38.0,8.662321e+17,DrTeckKhong,Sat May 30 16:46:20 +0000 2009,62208.0,4773.0,6282.0,242.0,139670.0
2,866236815463862272,Sun May 21 10:18:45 +0000 2017,yesterday was th european maritime day today i...,Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,NaN,21.0,8.662204e+17,EU_MARE,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0
3,866236868404314112,Sun May 21 10:18:57 +0000 2017,thesnp depute leader and candidate for moray a...,Ian Hudghton,HudghtonSNP,308677137,SNP President 2005 - 2020. SNP Member of the E...,Scotland,False,en,...,NaN,6.0,8.662227e+17,erikgeddes,Tue May 31 20:22:38 +0000 2011,21.0,5044.0,398.0,205.0,1789.0
4,866233710714257408,Sun May 21 10:06:25 +0000 2017,cacophonic that and fox hunting and dementia t...,Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,25067514.0,0.0,NaN,NaN,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0


In [21]:
my_stop_words = ['wonderful', 'did', 'didnt', 'issue', 'issues', 'actually', 'id','th',
                'community']

In [22]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_stop_words)

In [23]:
tv_30docs = TfidfVectorizer(stop_words = my_stop_words, min_df=30)
dtm_tv_30docs = tv_30docs.fit_transform(list(df['text']))

In [24]:
dtm_tv_30docs.shape

(3189504, 55165)

In [36]:
nmf_topics = NMF(n_components=50, init='random', max_iter=1000)
nmf_17k_words = nmf_topics.fit_transform(dtm_tv_30docs)

In [37]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

In [38]:
output_17kwords = display_topics(nmf_topics, tv_30docs.get_feature_names_out(), 10,
                                          topic_names=None)


Topic  1
people, young, american, million, working, country, disabled, british, lives, voted

Topic  2
today, meet, remember, visit, earlier, ago, met, discuss, celebrate, meeting

Topic  3
government, federal, funding, uk, tory, shutdown, budget, auspol, reopen, needs

Topic  4
brexit, deal, theresa, mays, says, britain, hard, referendum, nodeal, tory

Topic  5
years, ago, service, past, old, remember, ive, lost, come, lives

Topic  6
support, campaign, proud, pleased, delighted, strong, funding, continue, fully, continued

Topic  7
want, say, hear, thats, email, come, doesnt, sorry, leave, stay

Topic  8
live, watch, tune, facebook, hearing, video, debate, speech, ill, follow

Topic  9
news, fantastic, excellent, bbc, fake, brilliant, welcome, sad, breaking, thoughts

Topic  10
minister, prime, says, foreign, said, ministers, trade, question, shadow, canada

Topic  11
forward, looking, look, working, seeing, tomorrow, meeting, joining, hearing, soon

Topic  12
work, hard, doing, con

In [39]:
total = 0
for i in range(50):
    print(f'topic{i+1} {sum(nmf_topics.components_[i])}')
    total += sum(nmf_topics.components_[i])
total

topic1 9.471260030813879
topic2 9.795740431397094
topic3 13.581258189042478
topic4 18.17638695076209
topic5 15.8576440927775
topic6 12.078181726979217
topic7 9.37381318257204
topic8 16.71393304857601
topic9 11.230131043900506
topic10 26.43697145957931
topic11 13.290059475352537
topic12 14.44644481254399
topic13 14.355405858935764
topic14 16.05625947724107
topic15 30.63219788079049
topic16 17.271144171192383
topic17 18.92279032399891
topic18 24.815198707983573
topic19 22.16309219316302
topic20 14.28196840924359
topic21 16.03208445392887
topic22 21.35536076589633
topic23 18.204154720850784
topic24 18.700705618413295
topic25 13.489979210669976
topic26 19.479202649208748
topic27 19.675983822181585
topic28 12.016335587062542
topic29 20.174590074260735
topic30 24.486432134764804
topic31 27.621047074438003
topic32 17.506553577617684
topic33 16.18509636181709
topic34 27.151138218049237
topic35 32.63989243799144
topic36 20.680492670798397
topic37 15.122632439296968
topic38 17.34034707902662
top

1027.2976972634315